# Ingeniería de atributos

En este apartado se trabajará de la siguiente forma. En primer lugar, realizamos las transformaciones necesarias sobre el target. Posteriormente, se realizan las modificaciones sobre las variables predictoras. 

Dicho esto, pasamos a cargar las librerías que vamos a utilizar y la base de datos.

In [1]:
import pandas as pd 
import numpy as np # numpy y pandas to data wrangling 
from datetime import datetime, timedelta # to work w date
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from google.colab import drive

ModuleNotFoundError: No module named 'google'

In [4]:
aux = 'https://drive.google.com/file/d/1GWaKdMRk4Fx9qGgBMioNeO3E7_I3bA8k/view?usp=sharing'
link ='https://drive.google.com/uc?id=' + aux.split('/')[-2]
df = pd.read_csv(link)

In [5]:
df.columns

Index(['Date', 'Volume USDT_btc', 'cryptocurrency_adjusted', 'crypto_adjusted',
       'bitcoin_adjusted', 'bitcoin price_adjusted', 'ethereum_adjusted',
       'ethereum price_adjusted', 'stock market_adjusted',
       'wall street_adjusted', 'interest rate_adjusted', 'fed_adjusted',
       'bankruptcy_adjusted', 'china_adjusted', 'united states_adjusted',
       'war_adjusted', 'russia_adjusted', 'cryptocurrency_top_adjusted',
       'bitcoin_top_adjusted', 'investment_top_adjusted',
       'ethereum_top_adjusted', 'exchange_top_adjusted',
       'bankrup_top_adjusted', 'stock_market_top_adjusted',
       'inflation_adjusted', 'taxes_adjusted', 'digital_wallet_top_adjusted',
       'covid19_adjusted', 'pandemic_adjusted', 'elon musk_adjusted',
       'do kwon_adjusted', 'future_commodities_adjusted', 'banking_adjusted',
       'foreign_currency_adjusted', 'unix_eth', 'open_eth', 'high_eth',
       'low_eth', 'close_eth', 'Volume ETH_eth', 'Volume USDT_eth',
       'eth_close_open_mea

## Target 

In [4]:
# armado de variable target y conjunto de variables predictoras 
for i in range(1,8):
  df[f'log_y_lag{i}'] = np.log10(df[f'y_lag{i}']+1)
  df = df.drop(columns=[f'y_lag{i}'])
df

,Date,Volume USDT_btc,cryptocurrency_adjusted,crypto_adjusted,bitcoin_adjusted,bitcoin price_adjusted,ethereum_adjusted,ethereum price_adjusted,stock market_adjusted,wall street_adjusted,...,low_btc,close_btc,Volume BTC_btc,log_y_lag1,log_y_lag2,log_y_lag3,log_y_lag4,log_y_lag5,log_y_lag6,log_y_lag7
0,2015-08-08,3.801000,0.0,0.48,1.74,0.27,0.00,0.00,1.45,8.32,...,262.650000,263.000000,1.032690e+03,0.352183,0.350248,0.441695,0.498311,0.451018,0.363612,0.422426
1,2015-08-11,0.943500,0.0,0.88,1.89,0.49,0.00,0.00,2.61,9.92,...,261.610000,269.260000,2.474900e+02,0.350248,0.441695,0.498311,0.451018,0.363612,0.422426,0.401401
2,2015-08-12,2.558000,0.0,0.87,1.83,0.47,0.00,0.00,3.48,10.40,...,263.330000,265.410000,6.821900e+02,0.441695,0.498311,0.451018,0.363612,0.422426,0.401401,0.382017
3,2015-08-13,3.189000,0.0,0.87,1.95,0.35,0.00,0.00,2.61,10.40,...,260.000000,263.000000,8.389900e+02,0.498311,0.451018,0.363612,0.422426,0.401401,0.382017,0.400538
4,2015-08-14,2.129000,0.0,0.83,1.68,0.32,0.00,0.00,2.32,10.88,...,255.000000,265.000000,5.562300e+02,0.451018,0.363612,0.422426,0.401401,0.382017,0.400538,0.417472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2494,2022-06-10,247.612626,5.0,6.46,8.41,8.40,9.10,7.56,18.00,6.48,...,28863.371181,29091.044620,7.291809e+06,3.203747,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672
2495,2022-06-11,177.801516,5.0,7.14,8.99,9.80,15.34,16.74,8.70,5.94,...,28120.500000,28408.993958,5.092090e+06,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881
2496,2022-06-12,263.103277,7.2,8.16,11.02,11.20,15.34,15.39,7.20,6.03,...,26578.603357,26600.000000,7.240280e+06,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881,3.026502
2497,2022-06-13,1137.699906,14.6,15.98,28.13,35.00,26.00,27.00,30.00,7.38,...,21965.104691,22486.877767,2.748774e+07,3.082769,3.087707,3.062070,3.032672,3.017881,3.026502,3.053050


## Predictores

### Info de ethereum y bitcoin

#### Precios presentes

In [5]:
# ethereum
df['log_price_eth'] = np.log10(df['close_eth']+1)
# bitcoin
df['log_price_btc'] = np.log10(df['close_btc']+1)

##### Prueba de causalidad de granger 
Para testear si el precio del bitcoin sirve para predecir el precio futuro de ethereum.

In [6]:
# precio del bitcoin como predictor del precio futuro de ethereum 
t = grangercausalitytests(df[['log_y_lag1', 'log_price_btc']], maxlag=7)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1901  , p=0.6628  , df_denom=2495, df_num=1
ssr based chi2 test:   chi2=0.1904  , p=0.6626  , df=1
likelihood ratio test: chi2=0.1904  , p=0.6626  , df=1
parameter F test:         F=0.1901  , p=0.6628  , df_denom=2495, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=81.8773 , p=0.0000  , df_denom=2492, df_num=2
ssr based chi2 test:   chi2=164.0832, p=0.0000  , df=2
likelihood ratio test: chi2=158.9172, p=0.0000  , df=2
parameter F test:         F=81.8773 , p=0.0000  , df_denom=2492, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=57.5286 , p=0.0000  , df_denom=2489, df_num=3
ssr based chi2 test:   chi2=173.0713, p=0.0000  , df=3
likelihood ratio test: chi2=167.3347, p=0.0000  , df=3
parameter F test:         F=57.5286 , p=0.0000  , df_denom=2489, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=38.3585 , p=0.

In [7]:
# calculamos los rezagos de los 6 días anteriores al momento presente 
for i in range(1,8):
  df[f'log_price_btc_lag{i}'] = df['log_price_btc'].shift(i)

In [8]:
df[['Date','log_price_btc','log_price_btc_lag1','log_price_btc_lag2', 'log_price_btc_lag3', 'log_price_btc_lag4','log_price_btc_lag5','log_price_btc_lag6','log_price_btc_lag7']]

,Date,log_price_btc,log_price_btc_lag1,log_price_btc_lag2,log_price_btc_lag3,log_price_btc_lag4,log_price_btc_lag5,log_price_btc_lag6,log_price_btc_lag7
0,2015-08-08,2.421604,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-08-11,2.431782,2.421604,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-08-12,2.425551,2.431782,2.421604,NaN,NaN,NaN,NaN,NaN
3,2015-08-13,2.421604,2.425551,2.431782,2.421604,NaN,NaN,NaN,NaN
4,2015-08-14,2.424882,2.421604,2.425551,2.431782,2.421604,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2494,2022-06-10,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,4.475200,4.472764
2495,2022-06-11,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,4.475200
2496,2022-06-12,4.424898,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867
2497,2022-06-13,4.351948,4.424898,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540


##### Prueba de Dickey-Fuller aumentada
Para testear la existencia de raíz unitaria y comprobar si deberíamos agregar el precio rezagado de la variable dependiente como predictor

In [9]:
# ethereum
adfuller(df['log_y_lag1'])[1] 

0.3144684796254814

#### Volumen de transacciones

In [10]:
df

,Date,Volume USDT_btc,cryptocurrency_adjusted,crypto_adjusted,bitcoin_adjusted,bitcoin price_adjusted,ethereum_adjusted,ethereum price_adjusted,stock market_adjusted,wall street_adjusted,...,log_y_lag7,log_price_eth,log_price_btc,log_price_btc_lag1,log_price_btc_lag2,log_price_btc_lag3,log_price_btc_lag4,log_price_btc_lag5,log_price_btc_lag6,log_price_btc_lag7
0,2015-08-08,3.801000,0.0,0.48,1.74,0.27,0.00,0.00,1.45,8.32,...,0.422426,0.380211,2.421604,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-08-11,0.943500,0.0,0.88,1.89,0.49,0.00,0.00,2.61,9.92,...,0.401401,0.322219,2.431782,2.421604,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-08-12,2.558000,0.0,0.87,1.83,0.47,0.00,0.00,3.48,10.40,...,0.382017,0.376577,2.425551,2.431782,2.421604,NaN,NaN,NaN,NaN,NaN
3,2015-08-13,3.189000,0.0,0.87,1.95,0.35,0.00,0.00,2.61,10.40,...,0.400538,0.498311,2.421604,2.425551,2.431782,2.421604,NaN,NaN,NaN,NaN
4,2015-08-14,2.129000,0.0,0.83,1.68,0.32,0.00,0.00,2.32,10.88,...,0.417472,0.498311,2.424882,2.421604,2.425551,2.431782,2.421604,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2494,2022-06-10,247.612626,5.0,6.46,8.41,8.40,9.10,7.56,18.00,6.48,...,3.032672,3.221142,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,4.475200,4.472764
2495,2022-06-11,177.801516,5.0,7.14,8.99,9.80,15.34,16.74,8.70,5.94,...,3.017881,3.185397,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,4.475200
2496,2022-06-12,263.103277,7.2,8.16,11.02,11.20,15.34,15.39,7.20,6.03,...,3.026502,3.157361,4.424898,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867
2497,2022-06-13,1137.699906,14.6,15.98,28.13,35.00,26.00,27.00,30.00,7.38,...,3.053050,3.083189,4.351948,4.424898,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540


In [11]:
df['log_volume_btc-usd'] = np.log10(df['Volume USDT_btc']+1)
df['log_volume_eth-usd'] = np.log10(df['Volume USDT_eth']+1)

#### Variación interdiaria precios 

In [12]:
# ethereum
df['var_price_eth'] = df['close_eth'].div(df['close_eth'].shift(1))-1
# bitcoin
df['var_price_btc'] = df['close_btc'].div(df['close_btc'].shift(1))-1

In [13]:
df = df.dropna()
df

,Date,Volume USDT_btc,cryptocurrency_adjusted,crypto_adjusted,bitcoin_adjusted,bitcoin price_adjusted,ethereum_adjusted,ethereum price_adjusted,stock market_adjusted,wall street_adjusted,...,log_price_btc_lag2,log_price_btc_lag3,log_price_btc_lag4,log_price_btc_lag5,log_price_btc_lag6,log_price_btc_lag7,log_volume_btc-usd,log_volume_eth-usd,var_price_eth,var_price_btc
7,2015-08-17,0.440500,0.0,0.85,1.83,0.45,0.00,0.00,2.32,10.08,...,2.428135,2.424882,2.421604,2.425551,2.431782,2.421604,0.158513,1.435526,0.357143,0.021849
8,2015-08-18,7.226000,0.0,0.72,2.04,0.45,0.00,0.00,2.61,9.76,...,2.411216,2.428135,2.424882,2.421604,2.425551,2.431782,0.915189,0.785187,0.000000,-0.043336
9,2015-08-19,28.740000,0.0,0.84,2.61,0.69,0.00,0.00,2.61,10.24,...,2.420566,2.411216,2.428135,2.424882,2.421604,2.425551,1.473341,2.188169,-0.144737,-0.095618
10,2015-08-20,8.178000,0.0,0.84,2.19,0.55,0.00,0.00,3.77,10.24,...,2.401401,2.420566,2.411216,2.428135,2.424882,2.421604,0.962748,2.558204,0.330769,0.030837
11,2015-08-21,11.300000,0.0,0.81,2.04,0.51,0.00,0.00,7.54,11.20,...,2.357935,2.401401,2.420566,2.411216,2.428135,2.424882,1.089905,2.484585,-0.132948,-0.003718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2494,2022-06-10,247.612626,5.0,6.46,8.41,8.40,9.10,7.56,18.00,6.48,...,4.480211,4.493176,4.496540,4.475867,4.475200,4.472764,2.395523,3.659348,-0.070400,-0.033629
2495,2022-06-11,177.801516,5.0,7.14,8.99,9.80,15.34,16.74,8.70,5.94,...,4.478630,4.480211,4.493176,4.496540,4.475867,4.475200,2.252371,3.952189,-0.079057,-0.023445
2496,2022-06-12,263.103277,7.2,8.16,11.02,11.20,15.34,15.39,7.20,6.03,...,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,2.421774,3.988340,-0.062556,-0.063677
2497,2022-06-13,1137.699906,14.6,15.98,28.13,35.00,26.00,27.00,30.00,7.38,...,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540,3.056409,4.564969,-0.157108,-0.154629


In [14]:
# guardar el dataset en carpeta data set del proyecto en google drive
drive.mount('drive')
df.to_csv('dataset.csv')
!cp dataset.csv "/content/drive/MyDrive/ds/proyecto/dataset/"

Mounted at drive
